In [87]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output
import threading
import turtle
import scipy.integrate as spi
import warnings
import math
warnings.filterwarnings('ignore')
from IPython.display import display, clear_output

In [32]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [91]:
C_X0 = 0.1  # gX/L or 1e-4 mass fraction
C_S0 = 100  # gS/L or 0.1 mass fraction
Y = 0.4     # kgX/kgS
mu_max_hour = 0.1  # 1/h
k_d_hour = 0.01 # 1/h
K_S = 3.76    # kgS/m3
C_S_in = 1560 # kg/m3 (pure glucose in)
V_0 = 202   # ml
F_0 = 0     # ml/s
activation = 0
set_point = 0.25     # kg/m3
Yp = 0.15 #kgP/kgS
Ypace = 0.15 #kgP/kgS
C_P0 = 0 #gP/L
C_Pace0 = 0 #gP/L

In [196]:
def BioReactor(tf,C_X0,C_S0,Y,mu_max_hour,k_d_hour,K_S,C_S_in,V_0,F_0,activation, Fin, t_act, Yp, C_P0, C_Pace0,Ypace,pH0):
    t0 = 0
    dt = 1
    tf=tf*3600
    molarmass_acid=60.05
    Ka_acid=1.8*10**-5
    C_X = C_X0
    C_S = C_S0
    C_P =C_P0
    C_Pace =C_Pace0
    pH =pH0
    t = t0
    V = V_0/10**6
    F = F_0/10**6
    mu_max=mu_max_hour/3600
    k_d=k_d_hour/3600

    C_X_v, C_S_v, t_v, V_v, F_v, activation_v, C_P_v, C_Pace_v, pH_v = [C_X], [C_S], [t], [V], [F], [activation],[C_P],[C_Pace],[pH]
    mu_v, rX_v, rS_v, rXg_v, rXd_v = [], [], [], [], []
    while t < tf:
        
        # Check if there is activation of the pump
        if t % (t_act*3600) == 0 :
            activation = 1
            F  = Fin/10**6  # kg/m3/s
        else:
            activation = 0
            F = 0

        # Compute rates
        mu = mu_max*C_S/(C_S+K_S)
        rXg = mu * C_X
        rXd = k_d * C_X
        rX = rXg - rXd
        rS = -1/Y * rXg
        rP = -Yp * rS
        rPace = -Ypace * rS

        # Compute derivatives
        dXdt = rX-F/V*C_X
        dSdt = F/V*(C_S_in-C_S)+rS
        dVdt = F
        dPdt = rP-F/V*C_P
        dPacedt = rPace-F/V*C_Pace
        
        
        # Update results
        t += dt
        C_X += dXdt*dt
        C_S += dSdt*dt
        V += dVdt*dt
        C_P += dPdt*dt
        C_Pace += dPacedt*dt
        C_ph=C_Pace/molarmass_acid
        sqrt1=math.sqrt(10**-7+(Ka_acid*C_ph))
        pH=-1*math.log(sqrt1)
        if C_Pace==0:
            pH=7
        else:
            pass

        # Store results
        C_X_v.append(C_X)
        C_S_v.append(C_S)
        C_P_v.append(C_P)
        C_Pace_v.append(C_Pace)
        pH_v.append(pH)
        t_v.append(t)
        V_v.append(V*10**6)
        F_v.append(F*10**6)                                    
        mu_v.append(mu)
        rS_v.append(rS)
        rX_v.append(rX)
        rXg_v.append(rXg)
        rXd_v.append(rXd)
        activation_v.append(activation)
    #mu_v=mu_v[::3600]
    t_plot = [t/24/3600 for t in t_v]
    mu_plot = [mu*3600 for mu in mu_v]
    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(15,15))
    ax1.plot(t_plot,C_S_v,'r-',t_plot,C_X_v, t_plot,C_P_v,t_plot,C_Pace_v,'g-',linewidth=2)
    ax1.legend(['C_S','C_X','C_P','C_acetate'])
    ax1.set(xlabel='Time (days)', ylabel='Concentration (kg/$m^3$)')

    ax2.plot(t_plot, pH_v,linewidth=2)
    ax2.set(xlabel='Time (days)', ylabel='pH')
    ax2.legend(['pH of reactor'])

    ax3.plot(t_plot[:-1],mu_plot,'y-',linewidth=2)
    ax3.set(xlabel='Time (days)', ylabel='$\mu$ (1/h)')

    lns1 = ax4.plot(t_plot,F_v,linewidth=2,label='Flowrate') 
    ax4.set(xlabel='Time (days)', ylabel='Flowrate (ml/s)')
    ax5 = ax4.twinx()
    lns2 = ax5.plot(t_plot[1:],V_v[1:],'g-',linewidth=2,label='Volume')
    ax5.set_ylabel('Volume (mL)')
    ax4.ticklabel_format(axis="y", style='sci', scilimits=(0,0))
    lns = lns1+lns2
    labs = [l.get_label() for l in lns]
    ax4.legend(lns,labs,loc=0)
    print(math.sqrt(Ka_acid*C_ph))


In [193]:
file = open("Project_nino.jpg", "rb")
image = file.read()

image_headline = widgets.Image(
                    value=image,
                    format='jpg',
                    width='250'
                )
display(image_headline)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00\xc0\x00\xc0\x00\x00\xff\xe1\x00"Exif\x00\x00MM\…

In [197]:
style = {'description_width': 'initial'}

In [198]:
Bioreactor_final = widgets.interact(BioReactor, tf=widgets.BoundedFloatText(value=24, min=0, max=3600*24*10,step=1, description='Total time (h):',style=style, disabled=False), C_X0=widgets.BoundedFloatText(value=50, min=0, max=100.0,step=0.1, description='Biomass concentration (g/L):',style=style, disabled=False), C_S0=widgets.BoundedFloatText(value=0, min=0, max=100.0,step=0.1, description='Concentration glucose (g/L):',style=style, disabled=False), Y=widgets.BoundedFloatText(value=0.4, min=0, max=1.0,step=0.1, description='Yield:',style=style, disabled=False), mu_max_hour=widgets.BoundedFloatText(value=0.1, min=0, max=10.0,step=0.01, description='mu_max (1/h):',style=style, disabled=False), K_S=widgets.BoundedFloatText(value=3.6, min=0, max=10.0,step=0.01, description='K_S (kg s/m3):',style=style, disabled=False), C_S_in=widgets.BoundedFloatText(value=800, min=0, max=910,step=1, description='glucose concentration in feed (kg/m3):',style=style, disabled=False), V_0=widgets.BoundedFloatText(value=50, min=0, max=100,step=1, description='Volume (ml):',style=style, disabled=False), F_0=widgets.BoundedFloatText(value=0, min=0, max=5,step=0.1, description='Feed volume (ml):',style=style, disabled=False), k_d_hour=widgets.BoundedFloatText(value=0.01, min=0, max=0.1,step=0.001, description='K_d (1/h):',style=style, disabled=False), Fin=widgets.BoundedFloatText(value=0.5, min=0, max=5,step=0.01, description='Feed in (ml):',style=style, disabled=False), t_act=widgets.BoundedFloatText(value=4, min=0, max=20,step=0.1, description='Activation interval (h):',style=style, disabled=False), activation=0, Yp=widgets.BoundedFloatText(value=0.4, min=0, max=1.0,step=0.1, description='Yield (P/S):',style=style, disabled=False), C_P0=widgets.BoundedFloatText(value=0, min=0, max=100.0,step=0.1, description='Concentration ethanol (g/L):',style=style, disabled=False),Ypace=widgets.BoundedFloatText(value=0.4, min=0, max=1.0,step=0.1, description='Yield (actetate/S):',style=style, disabled=False), C_Pace0=widgets.BoundedFloatText(value=0, min=0, max=100.0,step=0.1, description='Concentration acetate (g/L):',style=style, disabled=False), pH0=widgets.BoundedFloatText(value=7, min=0, max=14.0,step=0.1, description='pH:',style=style, disabled=False) );

interactive(children=(BoundedFloatText(value=24.0, description='Total time (h):', max=864000.0, step=1.0, styl…